In [2]:
from Bio import SeqIO, Entrez
import os
from urllib.error import HTTPError
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from itertools import permutations, product
from sklearn.metrics import accuracy_score, confusion_matrix, balanced_accuracy_score
import tqdm
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split, cross_val_score
import inspect
from numpy import mean
from numpy import std
import pickle
from os import path
from sklearn.model_selection import cross_val_score
from warnings import simplefilter
from collections import OrderedDict
from sklearn.metrics import accuracy_score, auc, confusion_matrix, balanced_accuracy_score, precision_recall_curve, auc, roc_curve, roc_auc_score
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import BalancedBaggingClassifier
import requests
from sklearn.model_selection import GridSearchCV
import re
if (os.path.abspath('').split('/')[-1] == 'project'):
    %cd utils
elif (os.path.abspath('').split('/')[-1] == 'validate'):
    %cd ../utils

import query_utils
import model_utils
import validation_utils
import data_utils


print(os.path.abspath(''))

if (os.path.abspath('').split('/')[-1] == 'utils'):
    %cd ..

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action='ignore', category=FutureWarning)

Entrez.tool = "Zoonosis predictor"

Entrez.email = input("Enter an email address to use NCBI e-utils: ")

/Users/benjaminli/Documents/coding/scires/project/utils
/Users/benjaminli/Documents/coding/scires/project/utils
/Users/benjaminli/Documents/coding/scires/project


In [3]:
df = pd.read_csv('data/dataset1/human_infecting_virus', delimiter='\t', header=None)
df[['ID', 'DNA Sequence']] = df[0].str.split(expand=True)
df = df.drop(0, axis=1)
df['isZoonotic'] = 1


df2 = pd.read_csv('data/dataset1/Other_viruses', delimiter='\t', header=None)
df2[['ID', 'DNA Sequence']] = df2[0].str.split(expand=True)
df2 = df2.drop(0, axis=1)
df2['isZoonotic'] = 0


nardus = pd.read_csv('data/dataset2/nardus_sequences.csv')
nardus.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
df2.reset_index(drop=True, inplace=True)

taxIds = []
x = 0

cleaned_data = pd.read_csv('data/dataset2/FinalData_Cleaned.csv')
print(len(cleaned_data))
for name in tqdm.tqdm(cleaned_data['UniversalName']):
    # print("getting", name)
    url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=taxonomy&term={name}"
    try:
        response = requests.get(url)
    except HTTPError as e:
        print(e)
        break

    # Extract the taxonomy ID from the search results
    search_results = response.text
    # print(search_results)

    # search for ID tag within search results
    taxon_id = re.search(r"<Id>(\d+)</Id>", search_results).group(1)
    # print(taxon_id)
    taxIds.append(taxon_id)
    time.sleep(1)


mergedDf = pd.concat([df, df2, nardus], axis=0, ignore_index=True)
mergedDf = mergedDf.drop_duplicates(subset=['DNA Sequence'])
mergedDf.reset_index(drop=True, inplace=True)

print(mergedDf)
# retrieve tax ids from ncbi accession numbers and add to dataframe

# mergedDf = mergedDf.dropna(subset=['taxId'])
mergedDf.reset_index(drop=True, inplace=True)

913


100%|██████████| 913/913 [17:51<00:00,  1.17s/it]


                ID                                       DNA Sequence  \
0            37121  cttttaacaatcatagttttataaaagggtgtaaccgaagcggttt...   
1           129724  atactttacaattatcttgtaaaaagtagggtgtaaccgaaaaggg...   
2            59303  atggcggacgtgtgacatcaccgttcgctctttctaggatcctttg...   
3           120686  ttgttctacttcttactcattattataaattataatgtttgtataa...   
4            99565  gtgaatgatgatggcgtcgaaagacgtcgttgcagctactgctgct...   
...            ...                                                ...   
10622  NC_007661.1  gttaaaactctcacacttatggtggaactggatctgcaaaaatggg...   
10623  NC_007662.1  gttaaaaatctggttgtatctactcttgaatgaacgtgcataaagc...   
10624  NC_007663.1  gttaaaatctggaaccaatatggaagggatttatgcacgtgctttt...   
10625  NC_007664.1  gttaaaaaagagtgcagatgtcgagaatagtcttactaacaccagg...   
10626  NC_007665.1  gttaaaaacctcaagatgcatgccgctattacttcaatgaaacgtg...   

       isZoonotic  
0               1  
1               1  
2               1  
3               1  
4               1  
...

In [ ]:
# TODO: after adding tax ids to merged df, need to get ids of viruses like ebola
mergedDf['taxId'] = taxIds

In [3]:
def getSequences(mergedDf):
    accession_list = [] # maintain order
    isZoonotic_list = [] # maintain order
    accession_set = set()
    isZoonotic_set = set()


    for row in tqdm.tqdm(mergedDf.itertuples()):
        # row[13] = accession, row[15] = infects humans
        for single_acc in row[14].split("; "):
            # print(single_acc)
            if single_acc not in accession_set:
                accession_list.append(single_acc)
                isZoonotic_list.append(0 if not row[16] else 1)
                # accession_set.add(single_acc)
                # isZoonotic_set.add(row[15])
                # print(0 if not row[16] else 1)

    print("passed local retrieval")
    # TODO: RUN MULTIPLE THREADS TO SPEED UP
    threads = []
    vals = []

    
    for index, ID in enumerate(tqdm.tqdm(accession_list[:1000])): #only read the first 100 lol
        # multithread for speed up
        query_utils.queryKmer(ID, isZoonotic_list, index, vals)
        # x = threading.Thread(target=queryKmer, args=(ID, isZoonotic_list, index, vals))
        # threads.append(x)
        # x.start()

    # for index, thread in enumerate(tqdm.tqdm(threads)):
    #     thread.join()
    df = pd.DataFrame(vals)
    df.to_csv("data/nardus_sequences.csv")

    return df
    


In [7]:
def getSingleSequence(accessionID):
    try:
        QueryHandle = Entrez.efetch(db="nucleotide", id=accessionID, 
                                    rettype="gb", retmode="text")
    except HTTPError as Error:
        if Error.code == 400:  # Bad request
            raise ValueError(f"Accession number not found: {accessionID}")
        else:
            raise

    SeqRec = SeqIO.read(QueryHandle, "genbank")
    print(SeqRec.seq)
    X_info = SeqRec.seq.lower()
    kmer = 4
    s = product('acgt',repeat = kmer)
    permset = set(["".join(x) for x in list(s)])

    oDict = data_utils.assign_kmers_to_dict(X_info, permset, kmer) # convert ordereddict to pandas dataframe

    kmer_df = pd.DataFrame()

    for i in oDict:
        kmer_df.at[0, i]=oDict[i]
    # print(best_gradBoost.predict_proba(kmer_df))
    kmer_df = kmer_df.apply(lambda x: (x-x.min())/(x.max()-x.min()), axis=1)

    model = pickle.load(open('models/curr_models/xgBoost-f2-4.pkl', 'rb'))
    
    # re-align columns
    # print(inspect.getmembers(model))
    # kmer_df = kmer_df[model.train_meta_features]
    print(model.predict_proba(kmer_df)[:,1])


getSingleSequence("NC_002549.1")
# west nile is ALREADY validated - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5658487/
getSingleSequence("MF797870")
# dengue need to remove
getSingleSequence("NC_001474")
# zika - DOES NOT have?? - check other strains though
getSingleSequence("MG645981")
# # salmonella - too large
# getSingleSequence("CP026379")

# nonzoonotic cattle thing - dataset does not include this?
getSingleSequence("NC_028246.1")

CGGACACACAAAAAGAAAGAAGAATTTTTAGGATCTTTTGTGTGCGAATAACTATGAGGAAGATTAATAATTTTCCTCTCATTGAAATTTATATCGGAATTTAAATTGAAATTGTTACTGTAATCACACCTGGTTTGTTTCAGAGCCACATCACAAAGATAGAGAACAACCTAGGTCTCCGAAGGGAGCAAGGGCATCAGTGTGCTCAGTTGAAAATCCCTTGTCAACACCTAGGTCTTATCACATCACAAGTTCCACCTCAGACTCTGCAGGGTGATCCAACAACCTTAATAGAAACATTATTGTTAAAGGACAGCATTAGTTCACAGTCAAACAAGCAAGATTGAGAATTAACCTTGGTTTTGAACTTGAACACTTAGGGGATTGAAGATTCAACAACCCTAAAGCTTGGGGTAAAACATTGGAAATAGTTAAAAGACAAATTGCTCGGAATCACAAAATTCCGAGTATGGATTCTCGTCCTCAGAAAATCTGGATGGCGCCGAGTCTCACTGAATCTGACATGGATTACCACAAGATCTTGACAGCAGGTCTGTCCGTTCAACAGGGGATTGTTCGGCAAAGAGTCATCCCAGTGTATCAAGTAAACAATCTTGAAGAAATTTGCCAACTTATCATACAGGCCTTTGAAGCAGGTGTTGATTTTCAAGAGAGTGCGGACAGTTTCCTTCTCATGCTTTGTCTTCATCATGCGTACCAGGGAGATTACAAACTTTTCTTGGAAAGTGGCGCAGTCAAGTATTTGGAAGGGCACGGGTTCCGTTTTGAAGTCAAGAAGCGTGATGGAGTGAAGCGCCTTGAGGAATTGCTGCCAGCAGTATCTAGTGGAAAAAACATTAAGAGAACACTTGCTGCCATGCCGGAAGAGGAGACAACTGAAGCTAATGCCGGTCAGTTTCTCTCCTTTGCAAGTCTATTCCTTCCGAAATTGGTAGTAGGAGAAAAGGCTTGCCTTGAGAAGGTTCAAAGGCAAATTCAA

KeyError: "None of [Index([                            (0.6855943739414215, 0.3144056357443333, 0.8168710499204848, 0.1831289500795154, 0.8435552080365321, 0.15644479196346797, 0.9, 0.1, 0.524, 0.476),\n                               (0.05981648564338684, 0.9401835143566132, 0.06020104652894218, 0.9397989534710579, 0.12637800851826914, 0.8736219914817308, 0.0, 1.0, 0.2384, 0.7616),\n                                  (0.1582379400730133, 0.8417620569467544, 0.35396513855760736, 0.6460348614423926, 0.33895588427654433, 0.6610441157234558, 0.4, 0.6, 0.336, 0.664),\n                                  (0.8338394939899445, 0.1661605046130717, 0.9764818362862524, 0.02351816371374761, 0.9556622641869525, 0.04433773581304741, 1.0, 0.0, 0.716, 0.284),\n                                   (0.2568675220012665, 0.7431324779987335, 0.5467728065088523, 0.4532271934911477, 0.01681408455252753, 0.9831859154474725, 0.5, 0.5, 0.432, 0.568),\n                              (0.9696400165557861, 0.030359988682

In [11]:

    # print(kmer_df)
    # best_gradBoost = pickle.load(open('models/nardus_testing/gradBoost.pkl', 'rb'))
    # cols_when_model_builds = best_gradBoost.feature_names_in_
    # kmer_df=kmer_df[cols_when_model_builds]
    
    # print(best_gradBoost.predict_proba(kmer_df)[:,1])
    # other = pickle.load(open('models/curr_models/gradBoost.pkl', 'rb'))
    
    # cols_when_model_builds = other.feature_names_in_
    # kmer_df=kmer_df[cols_when_model_builds]
    
    # print(other.predict_proba(kmer_df)[:,1])

    # asdaf = pickle.load(open('models/curr_models/knn.pkl', 'rb'))
    
    # print(asdaf.predict_proba(kmer_df)[:,1])

    # asdaf = pickle.load(open('models/curr_models/randforest.pkl', 'rb'))
    
    # print(asdaf.predict_proba(kmer_df)[:,1])